In [22]:
# 📄 載入 CSV 檔案
import pandas as pd

file_path = "nstc_jobs_partial.csv"
df = pd.read_csv(file_path)

# 檢查資料
df = df.dropna(subset=["職缺名稱", "發佈日期", "連結", "詳細內容"])
print(f"資料筆數：{len(df)}")
df.head()

資料筆數：298


,職缺名稱,發佈日期,連結,詳細內容
0,本會網站「求才訊息」免責聲明,2016-08-24,https://www.nstc.gov.tw/folksonomy/detail/b4a0...,(一)本會網站「求才訊息」內容由受本會補助之大專院校及研究機關(構)或其所屬之研究人員所刊登...
1,國立臺灣科技大學產學營運智財技轉中心-智權管理師一名(計畫人員),2025-07-22,https://www.nstc.gov.tw/folksonomy/detail/201c...,工作項目：\n1.工程、應科、人文學院下列案件業務辦理與收益分配\n專利授權、讓與、技術移轉...
2,臺中榮民總醫院 細胞治療與再生醫學中心實驗室 徵求專任研究助理2名,2025-07-22,https://www.nstc.gov.tw/folksonomy/detail/34c2...,【應徵要求】\n1. 工作經歷：不拘\n2. 國內、外大學生命科學、生物學、醫技系、醫藥衛生...
3,中國醫藥大學生化暨分子生物研究所 張瓊文老師實驗室博士後研究員,2025-07-22,https://www.nstc.gov.tw/folksonomy/detail/719b...,中國醫藥大學生化暨分子生物研究所 張瓊文老師實驗室博士後研究員\n【職缺名稱】國科會補助研究...
4,國立中正大學管理學院金融科技碩士學位學程誠徵專任教師,2025-07-22,https://www.nstc.gov.tw/folksonomy/detail/1c13...,一、徵聘職級及員額: 助理教授級(含)以上師資1名\n二、起聘日期: 民國 115年2月1日...


In [23]:
# 🧩 整合欄位：將職缺名稱、發佈日期與詳細內容合併為單一文字欄位
texts = [
    f"職缺名稱：{row['職缺名稱']}\n發佈日期：{row['發佈日期']}\n詳細內容：\n{row['詳細內容']}"
    for _, row in df.iterrows()
]

In [24]:
metadatas = [
    {
        "職缺名稱": row["職缺名稱"],
        "連結": row["連結"]
    }
    for _, row in df.iterrows()
]

In [25]:
import langchain
# 🔗 封裝成 LangChain 的 Document 格式
from langchain.docstore.document import Document

docs = [
    Document(page_content=text, metadata=meta)
    for text, meta in zip(texts, metadatas)
]


In [26]:
# ✂️ 切割長文本成多段 chunk
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
split_docs = splitter.split_documents(docs)

# 檢查切割後的段落
for i, doc in enumerate(split_docs[:3]):
    print(f"段落 {i+1}:")
    print(doc.page_content[:200] + "...\n")
    print(f"來自職缺：{doc.metadata['職缺名稱']}")
    print(f"原始連結：{doc.metadata['連結']}\n---\n")


段落 1:
職缺名稱：本會網站「求才訊息」免責聲明
發佈日期：2016-08-24
詳細內容：
(一)本會網站「求才訊息」內容由受本會補助之大專院校及研究機關(構)或其所屬之研究人員所刊登(以下簡稱訊息刊登者)，本會僅提供訊息轉載之資訊服務平台，不對刊登內容進行任何背書與保證。
(二)本會已要求訊息刊登者應提供詳實真確之資訊，並請訊息刊登者對其所刊登內容負完全責任，惟求才訊息使用人(以下簡稱訊息使用人)亦應負...

來自職缺：本會網站「求才訊息」免責聲明
原始連結：https://www.nstc.gov.tw/folksonomy/detail/b4a0bc0a-cdb8-4264-9ee6-cdfe68508d93?l=ch
---

段落 2:
(四)本會保留隨時更新本免責聲明的權利，任何更改於本網站發佈時，立即生效。請訊息使用人於每次瀏覽本網站「求才訊息」時，務必查看此免責聲明。如訊息使用人繼續使用本網站「求才訊息」，即代表訊息使用人同意接受更改後的免責聲明約束。...

來自職缺：本會網站「求才訊息」免責聲明
原始連結：https://www.nstc.gov.tw/folksonomy/detail/b4a0bc0a-cdb8-4264-9ee6-cdfe68508d93?l=ch
---

段落 3:
職缺名稱：國立臺灣科技大學產學營運智財技轉中心-智權管理師一名(計畫人員)
發佈日期：2025-07-22
詳細內容：
工作項目：
1.工程、應科、人文學院下列案件業務辦理與收益分配
專利授權、讓與、技術移轉案。
產學合作案含技術移轉金。
國科會先期技轉案上繳。
2.產學案、專利授權案分期應收權利金與衍生利益金之盤點與追蹤、分配。
3.協助校內外技轉績效、技專校院資料庫填報、校內技轉法規與合約修訂...

來自職缺：國立臺灣科技大學產學營運智財技轉中心-智權管理師一名(計畫人員)
原始連結：https://www.nstc.gov.tw/folksonomy/detail/201c6238-e246-4096-8ffe-2ef3440d95bf?l=ch
---



In [27]:
# 1️⃣ 導入模型
from sentence_transformers import SentenceTransformer

# 使用 MiniLM 模型（也可改為 paraphrase-multilingual-MiniLM-L12-v2）
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2️⃣ 將所有 chunk 的內容轉成 embedding 向量
texts_for_embedding = [doc.page_content for doc in split_docs]
embeddings = embedding_model.encode(texts_for_embedding, show_progress_bar=True)


Batches:   0%|          | 0/35 [00:00<?, ?it/s]

In [28]:
# 1️⃣ 建立 FAISS Index
import faiss
import numpy as np

embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings))

# 2️⃣ 保留 metadata 與 split_docs 做配對用
# 將每個 index 對應的 chunk 存進 list
indexed_docs = split_docs


In [32]:
def rag_query(query, top_k=5):
    query_embedding = embedding_model.encode([query])
    D, I = index.search(np.array(query_embedding), top_k)

    retrieved_docs = [indexed_docs[i] for i in I[0]]

    print(f"🔍 問題：{query}\n")
    print("📌 找到的相關職缺：\n")

    for i, doc in enumerate(retrieved_docs):
        title = doc.metadata["職缺名稱"]
        link = doc.metadata["連結"]
        snippet = doc.page_content.strip().replace("\n", " ")
        if len(snippet) > 200:
            snippet = snippet[:200] + "..."

        print(f"{i+1}. 🧾 {title}")
        print(f"🔗 連結：{link}")
        print(f"📝 摘要：{snippet}")
        print("---\n")


In [33]:
rag_query("有提供勞保的職缺有哪些？")

🔍 問題：有提供勞保的職缺有哪些？

📌 找到的相關職缺：

1. 🧾 銘傳大學電腦與通訊工程學系誠徵兼任講師(含)以上教師1名
🔗 連結：https://www.nstc.gov.tw/folksonomy/detail/06eb9d08-abe0-47ee-888a-97ff1f50b10f?l=ch
📝 摘要：十、其他說明： 1.(本徵才公告蒐集之個人資料，僅限於人才甄選作業使用，非經當事人同意，絕不轉做其他用途，亦不會公佈任何資訊，並遵循本校資料保存與安  全控管辦理(詳細個資管理可參閱「銘傳大學個人資料保護管理要點」與「銘傳大學個人資料保護專區」http://pims.mcu.edu.tw)。 發佈日期：2025-07-08 00:00:00
---

2. 🧾 銘傳大學資訊工程學系誠徵兼任講師(含)以上教師2名
🔗 連結：https://www.nstc.gov.tw/folksonomy/detail/99cf938a-08b6-4c91-b80f-d56e2f9336c6?l=ch
📝 摘要：十、其他說明： 1.(本徵才公告蒐集之個人資料，僅限於人才甄選作業使用，非經當事人同意，絕不轉做其他用途，亦不會公佈任何資訊，並遵循本校資料保存與安  全控管辦理(詳細個資管理可參閱「銘傳大學個人資料保護管理要點」與「銘傳大學個人資料保護專區」http://pims.mcu.edu.tw)。 發佈日期：2025-07-09 00:00:00
---

3. 🧾 銘傳大學資訊工程學系誠徵專任或專案助理教授以上教師1名
🔗 連結：https://www.nstc.gov.tw/folksonomy/detail/e93dd151-a979-402f-bad5-000b03f727e7?l=ch
📝 摘要：十、其他說明： 1.(本徵才公告蒐集之個人資料，僅限於人才甄選作業使用，非經當事人同意，絕不轉做其他用途，亦不會公佈任何資訊，並遵循本校資料保存與安  全控管辦理(詳細個資管理可參閱「銘傳大學個人資料保護管理要點」與「銘傳大學個人資料保護專區」http://pims.mcu.edu.tw)。 發佈日期：2025-07-09 00:00:00
---

4. 🧾 中國醫藥大學 癌症生物精準醫學研究中心  王紹椿老師實驗室 誠徵 博士後研究員
🔗 連結：https://